# Phase Space Plot

In [ ]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

channel_data = DSamp[0][0][1]

def delay_embedding(data, emb_dim, delay):
    N = len(data)
    return np.array([data[i:i+emb_dim*delay:delay] for i in range(N - emb_dim * delay + 1)])

def false_nearest_neighbors(data, emb_dim, delay, R=10):
    N = len(data)
    false_neighbors = np.zeros(emb_dim)

    for d in range(1, emb_dim + 1):
        emb_data = delay_embedding(data, d, delay)
        nbrs = NearestNeighbors(n_neighbors=2).fit(emb_data[:-delay])
        distances, indices = nbrs.kneighbors(emb_data[:-delay])
        neighbor_index = indices[:, 1]
        neighbor_distance = np.abs(data[neighbor_index + delay] - data[np.arange(N - d * delay) + delay])
        false_neighbors[d - 1] = np.mean((neighbor_distance / distances[:, 1]) > R)

    return false_neighbors

# Using this function:
def delay_embedding(data, emb_dim, delay):
    N = len(data)
    return np.array([data[i:i+emb_dim*delay:delay].flatten() for i in range(N - emb_dim * delay + 1)])

channel_data_flat = channel_data.flatten()
fnn = false_nearest_neighbors(channel_data_flat, emb_dim=10, delay=1)


# Plot the FNN as a function of embedding dimension
plt.figure()
plt.plot(np.arange(1, max_dim+1), fnn)
plt.xlabel('Embedding Dimension')
plt.ylabel('Fraction of False Nearest Neighbors')
plt.title('Estimation of Embedding Dimension using FNN Method')
plt.show()

In [ ]:
# Print the results
for dim, fnn_fraction in enumerate(fnn, 1):
    print(f"Embedding dimension: {dim}, Fraction of false nearest neighbors: {fnn_fraction:.2f}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Define a function for delay embedding
def delay_embedding(data, emb_dim, delay):
    N = len(data)
    embedded_data = np.zeros((N - (emb_dim - 1) * delay, emb_dim))
    for i in range(N - (emb_dim - 1) * delay):
        embedded_data[i] = [data[i + j * delay] for j in range(emb_dim)]
    return embedded_data

# Choose the first channel of your EEG data
channel_data = EEGdata[0, :]

# Perform delay embedding with embedding dimension 3 and delay 1
embedded_channel_data = delay_embedding(channel_data, emb_dim=3, delay=1)

# Create 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Change viewing angle
ax.view_init(elev=45, azim=21)

ax.plot(embedded_channel_data[:, 0], embedded_channel_data[:, 1], embedded_channel_data[:, 2])
plt.title('Phase Space Plot')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define a function for delay embedding
def delay_embedding(data, emb_dim, delay):
    N = len(data)
    embedded_data = np.zeros((N - (emb_dim - 1) * delay, emb_dim))
    for i in range(N - (emb_dim - 1) * delay):
        embedded_data[i] = [data[i + j * delay] for j in range(emb_dim)]
    return embedded_data

# Choose the first channel of your EEG data
channel_data = EEGdata[0, :]

# Perform delay embedding with embedding dimension 2 and delay 1
embedded_channel_data = delay_embedding(channel_data, emb_dim=2, delay=1)

# Create 2D plot
plt.figure()
plt.plot(embedded_channel_data[:, 0], embedded_channel_data[:, 1])
plt.title('Phase Space Plot')
plt.xlabel('Embedding Dimension 1')
plt.ylabel('Embedding Dimension 2')
plt.show()